In [8]:
import pandas as pd
import numpy as np
import os
import json
import joblib # .pkl dosyalarını kaydetmek için
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from datetime import datetime

# Dosya Yolları
DATA_PATH = "../data/lgs_full_data.csv"
MODEL_DIR = "../model/model_v1"

# Model klasörü yoksa oluştur
os.makedirs(MODEL_DIR, exist_ok=True)

print("🚀 Eğitim hazırlığı başladı...")

🚀 Eğitim hazırlığı başladı...


In [9]:
# 1. Veriyi Yükle
df = pd.read_csv(DATA_PATH)

# Veri temizliği: Eksik verileri doldur
df = df.fillna("")

# 2. Özellik (X) ve Hedef (y) Belirleme
# Soru metni ve şıkları birleştiriyoruz: "Soru metni... A) şık B) şık..."
df['Full_Text'] = df['Soru_Metni'] + " " + df['Secenek_A'] + " " + df['Secenek_B'] + " " + df['Secenek_C'] + " " + df['Secenek_D']

X = df['Full_Text']
y = df['Ana_Konu']

print(f"Eğitim için {len(df)} adet soru hazır.")

# 3. Label Encoding (Kategorileri Sayıya Çevirme)
# Örn: "Paragrafta Anlam" -> 0, "Dil Bilgisi" -> 1
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# 4. Vectorization (Metni Sayısal Matrise Çevirme)
# TF-IDF yöntemi kullanıyoruz (En sık geçen kelimelerin ağırlığını hesaplar)
vectorizer = TfidfVectorizer(max_features=2000, stop_words=None) # Türkçe stop words eklenebilir
X_vectorized = vectorizer.fit_transform(X)

print(f"Vektörleştirme tamamlandı. Özellik sayısı: {X_vectorized.shape[1]}")

Eğitim için 160 adet soru hazır.
Vektörleştirme tamamlandı. Özellik sayısı: 2000


In [10]:
from sklearn.svm import LinearSVC
import collections

# 1. Yetersiz Veriyi Temizleme
# Hangi konudan kaç tane var bakalım
topic_counts = collections.Counter(y) # y, orijinal metin listesidir
print("🧐 Konu Sayıları:", topic_counts)

# Sadece 1 tane örneği olan konuların İSİMLERİNİ bul
rare_topic_names = [label for label, count in topic_counts.items() if count < 2]

if rare_topic_names:
    print(f"⚠️ DİKKAT: Şu konular veriden çıkarılıyor (Sadece 1 kez sorulmuşlar): {rare_topic_names}")

    # İsimleri sayısal ID'ye çevir ki y_encoded içinden eleyebilelim
    rare_ids = label_encoder.transform(rare_topic_names)

    # Bu ID'leri bulup veriden atıyoruz
    mask = ~np.isin(y_encoded, rare_ids)
    X_filtered = X_vectorized[mask]
    y_filtered = y_encoded[mask]

else:
    print("✅ Tüm konulardan yeterli sayıda örnek var.")
    X_filtered = X_vectorized
    y_filtered = y_encoded

# 2. Veriyi Bölme (Stratify Artık Çalışır)
X_train, X_test, y_train, y_test = train_test_split(
    X_filtered,
    y_filtered,
    test_size=0.2,
    random_state=42,
    stratify=y_filtered # Artık hata vermeyecek
)

# 3. Model Eğitimi (Linear SVC)
model = LinearSVC(class_weight='balanced', random_state=42, dual='auto')
print("⏳ Model (SVM) eğitiliyor...")
model.fit(X_train, y_train)
print("✅ Eğitim tamamlandı!")

🧐 Konu Sayıları: Counter({'Paragrafta Anlam': 77, 'Sözel Mantık': 27, 'Dil Bilgisi': 17, 'Yazım Kuralları': 13, 'Sözcükte Anlam': 13, 'Grafik Yorumlama': 10, 'Fiilimsiler': 1, 'Cümle Türleri': 1, 'Söz Sanatları': 1})
⚠️ DİKKAT: Şu konular veriden çıkarılıyor (Sadece 1 kez sorulmuşlar): ['Fiilimsiler', 'Cümle Türleri', 'Söz Sanatları']
⏳ Model (SVM) eğitiliyor...
✅ Eğitim tamamlandı!


In [11]:
# --- DÜZELTİLMİŞ KOD BLOĞU ---

# Tahmin yap
y_pred = model.predict(X_test)

# Başarı skorları
accuracy = accuracy_score(y_test, y_pred)

# HATA ÇÖZÜMÜ:
# Bütün olası etiket indekslerini (0, 1, 2... 8) oluşturuyoruz.
all_labels = range(len(label_encoder.classes_))

report = classification_report(
    y_test,
    y_pred,
    labels=all_labels,                # Tüm etiketleri dikkate almasını söylüyoruz
    target_names=label_encoder.classes_,
    output_dict=True,
    zero_division=0                   # Test setinde hiç olmayan konular için hata verme, 0 yaz
)

print(f"🎯 Model Doğruluğu (Accuracy): %{accuracy*100:.2f}")

# İstatistik JSON'ı hazırla
training_stats = {
    "date": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
    "total_samples": len(df),
    "accuracy": accuracy,
    "algorithm": "RandomForestClassifier",
    "classes": list(label_encoder.classes_),
    "detailed_report": report
}

# JSON kaydet
stats_path = os.path.join(MODEL_DIR, "training_stats.json")
with open(stats_path, 'w', encoding='utf-8') as f:
    json.dump(training_stats, f, ensure_ascii=False, indent=2)

print(f"📊 İstatistikler kaydedildi: {stats_path}")

🎯 Model Doğruluğu (Accuracy): %65.62
📊 İstatistikler kaydedildi: ../model/model_v1/training_stats.json


In [12]:
# 1. Modeli Kaydet
joblib.dump(model, os.path.join(MODEL_DIR, "topic_classifier.pkl"))

# 2. Vektörleştiriciyi Kaydet (Yeni gelen metni aynı şekilde çevirmek için şart)
joblib.dump(vectorizer, os.path.join(MODEL_DIR, "vectorizer.pkl"))

# 3. Etiket Dönüştürücüyü Kaydet (Sayıları tekrar yazıya çevirmek için)
joblib.dump(label_encoder, os.path.join(MODEL_DIR, "label_encoder.pkl"))

print("\n💾 Tüm dosyalar başarıyla oluşturuldu:")
print(f"1. {os.path.join(MODEL_DIR, 'topic_classifier.pkl')}")
print(f"2. {os.path.join(MODEL_DIR, 'vectorizer.pkl')}")
print(f"3. {os.path.join(MODEL_DIR, 'label_encoder.pkl')}")


💾 Tüm dosyalar başarıyla oluşturuldu:
1. ../model/model_v1/topic_classifier.pkl
2. ../model/model_v1/vectorizer.pkl
3. ../model/model_v1/label_encoder.pkl


In [13]:
def predict_topic(text):
    # Kayıtlı araçları yükle (Normalde bunu API tarafında yaparız ama test için burada)
    vec = joblib.load(os.path.join(MODEL_DIR, "vectorizer.pkl"))
    mod = joblib.load(os.path.join(MODEL_DIR, "topic_classifier.pkl"))
    enc = joblib.load(os.path.join(MODEL_DIR, "label_encoder.pkl"))

    # 1. Vektörleştir
    text_vec = vec.transform([text])

    # 2. Tahmin et
    prediction_idx = mod.predict(text_vec)[0]

    # 3. Etikete çevir
    topic_name = enc.inverse_transform([prediction_idx])[0]

    return topic_name

# TEST
ornek_soru = "Fiilimsiler, fiil kök veya gövdelerinden belirli eklerle türetilen sözcüklerdir. Aşağıdakilerin hangisinde sıfat-fiil vardır?"
tahmin = predict_topic(ornek_soru)

print(f"\n🧪 TEST SORUSU: {ornek_soru}")
print(f"🤖 MODEL TAHMİNİ: {tahmin}")


🧪 TEST SORUSU: Fiilimsiler, fiil kök veya gövdelerinden belirli eklerle türetilen sözcüklerdir. Aşağıdakilerin hangisinde sıfat-fiil vardır?
🤖 MODEL TAHMİNİ: Dil Bilgisi
